In [3]:
!pip install googletrans

     |████████████████████████████████| 61kB 2.9MB/s 
     |████████████████████████████████| 931kB 7.6MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 102kB 8.8MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=c2dbaae9ef1b0efbb5f9fb92455b956568692f690ea10dd7dde9c58d660026fe
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=f1b0808e365868534b0fdbba8d4baebfb94fbf91c33109767a0d9a830793d753
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletrans contextvars


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import re
from textblob import TextBlob
import pandas as pd
from googletrans import Translator
import time
import numpy as np

In [6]:
data  = pd.read_csv('/content/drive/My Drive/data for project/juhi project data/tweet6_25.csv', parse_dates=['created_at'])

In [7]:
def clean_tweet(data, tweet):
  return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

def get_tweet_sentiment(data, tweet):
  analysis = TextBlob(clean_tweet(data, translate(tweet)))
  if (analysis.sentiment.polarity>0):
    return 'positive'
  elif (analysis.sentiment.polarity == 0):
    return 'neutral'
  else:
    return 'negative'

translator = Translator()
def translate(word):
  return translator.translate(word, dest='en').text

def get_tweets(data):
  tweets = []
  count = 0
  for tweet in data.full_text:
    parsed_tweet = {}
    parsed_tweet['text'] = tweet
    parsed_tweet['sentiment'] = get_tweet_sentiment(data, translate(tweet))
    tweets.append(parsed_tweet)
    count+=1
  return tweets

def user_id(df):
  df['user_id'] = pd.DataFrame((data.user.str.split(expand=True)[1]).str.split(",", expand=True)[0])
  df['user_id'] = df['user_id'].astype(int)
  df.sort_values('user_id', inplace=True, ignore_index=True)
  df = df[['user_id','id','full_text','lang','retweet_count']]
  a = df['user_id'].value_counts().index[df['user_id'].value_counts() >= 3]
  col = pd.DataFrame({'user_id':a})
  df = df.merge(col, left_on=None, right_on=None)
  df.sort_values('user_id', inplace = True, ignore_index=True)
  return df

In [8]:
data = user_id(data)

In [9]:
data.keys()

Index(['user_id', 'id', 'full_text', 'lang', 'retweet_count'], dtype='object')

In [10]:
var = get_tweets(data)

In [11]:
var

[{'sentiment': 'negative',
  'text': 'RT @CongressPositiv: बीमारी को महामारी बनने के लिए क्या सामग्री चाहिए? \nकुछ ज्यादा नहीं बस एक निकम्मी सरकार #बिहार \n\n@INCBihar @IYCBihar @N…'},
 {'sentiment': 'negative',
  'text': 'RT @CongressPositiv: बीमारी को महामारी बनने के लिए क्या सामग्री चाहिए? \nकुछ ज्यादा नहीं बस एक निकम्मी सरकार #बिहार \n\n@INCBihar @IYCBihar @N…'},
 {'sentiment': 'negative',
  'text': 'RT @CongressPositiv: बीमारी को महामारी बनने के लिए क्या सामग्री चाहिए? \nकुछ ज्यादा नहीं बस एक निकम्मी सरकार #बिहार \n\n@INCBihar @IYCBihar @N…'},
 {'sentiment': 'negative',
  'text': 'RT @CongressPositiv: बीमारी को महामारी बनने के लिए क्या सामग्री चाहिए? \nकुछ ज्यादा नहीं बस एक निकम्मी सरकार #बिहार \n\n@INCBihar @IYCBihar @N…'},
 {'sentiment': 'negative',
  'text': 'RT @CongressPositiv: बीमारी को महामारी बनने के लिए क्या सामग्री चाहिए? \nकुछ ज्यादा नहीं बस एक निकम्मी सरकार #बिहार \n\n@INCBihar @IYCBihar @N…'},
 {'sentiment': 'negative',
  'text': 'RT @CongressPositiv: बीमारी को महामार

In [12]:
df = pd.DataFrame(var)
df.head()

,text,sentiment
0,RT @CongressPositiv: बीमारी को महामारी बनने के...,negative
1,RT @CongressPositiv: बीमारी को महामारी बनने के...,negative
2,RT @CongressPositiv: बीमारी को महामारी बनने के...,negative
3,RT @CongressPositiv: बीमारी को महामारी बनने के...,negative
4,RT @CongressPositiv: बीमारी को महामारी बनने के...,negative


In [13]:
df.to_csv('Final_Sentiment_Analysis.csv')